#### Testing out Gibbs zig-zag code:

In [1]:
using Distributions, TimeIt, ProgressMeter, PyPlot

In [2]:
include("zz_structures.jl")
include("mbsampler.jl")

get_clustering (generic function with 1 method)

In [3]:
d, Nobs = 5, 100
p = 1e-1
X = rand(Normal(), d, Nobs) .* rand(Binomial(1,p), d, Nobs)
X[1,:] = ones(Nobs)
ξ_true = rand(Normal(),d).* rand(Binomial(1,0.5), d)
ξ_true[1] = rand(Normal())
y = [rand(Binomial(1, 1/(1+exp(-ξ_true'X[:,j]))), 1)[1] + 0. for j in 1:Nobs];
mb_size = 10
σ02 = 1

# prior = HS_prior(d, σ02)
prior = GDP_prior(d, σ02)
# prior = gaussian_prior_nh(d, σ02)

GDP_prior(5, 1.0, [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], 1.0, 1.0, 1.0, 1.0, 1.0)

In [4]:
my_ll = ll_logistic(X,y);
my_model = model(my_ll, prior)
root = find_root(my_model, rand(d));

In [5]:
# Sub-sampling with weights 
ϵ = 1e-2
weights = abs.(X) + ϵ
weights ./= sum(weights,2)
mb_size = 10;
gs = [wumbsampler(Nobs, mb_size, weights[i,:]) for i in 1:d]
gs_list = mbsampler_list(d,gs);

In [6]:
A = eye(d)
opf = projopf(A, 100, hyperparam_size(prior))
opt = maxa_opt(10^5)
outp = outputscheduler(opf,opt);

In [7]:
my_zz_sampler = zz_sampler(0, gs_list, linear_bound(my_model.ll, my_model.pr, gs_list))
hyper_sampler = block_gibbs_sampler(1.)
blocksampler = Array{msampler}(2)
blocksampler[1] = my_zz_sampler
blocksampler[2] = hyper_sampler;

In [8]:
start = time()
ZZ_block_sample(my_model::model, outp::outputscheduler, blocksampler::Array{msampler})
print( round((time()-start)/60, 2), " mins")

0.21 mins

In [9]:
n_samples = 10^4
xi_samples = extract_samples(outp.opf.xi_skeleton, 
                             outp.opf.bt_skeleton, 
                             outp.opf.bt_skeleton[end]/n_samples,
                             "linear");
hyper_samples = extract_samples(outp.opf.hyper_skeleton, 
                                outp.opf.bt_skeleton, 
                                outp.opf.bt_skeleton[end]/n_samples, 
                                "constant");

In [10]:
[compute_configT(my_model, xi_samples[:,1:end-1], hyper_samples[:,1:end-1], k) for k in 1:d]

5-element Array{Float64,1}:
 0.846718
 1.16419 
 1.03893 
 1.42396 
 1.16749 